In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
import os
import matplotlib.pyplot as plt

In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgbm

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, GaussianNoise
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


# 데이터 불러오기

In [4]:
# 데이터 불러오기
train = pd.read_csv('/home/lab21/data/train.csv', index_col=0)
train_2 = pd.read_csv('/home/lab21/data/train_x_0.2_99.8.csv', index_col=0)

test = pd.read_csv('/home/lab21/data/test.csv', index_col=0)
sample_submission = pd.read_csv('/home/lab21/data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
#x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = train_2.drop(columns=['fiberID'], axis=1)

test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [5]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [6]:
x=x.astype('float32')
test_x=test_x.astype('float32')

# Train_Test_SpIit

In [7]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, \
                                                    train_size=0.8, shuffle=True ,random_state=42)

# Scaler

In [8]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

# Variables

In [9]:
import datetime
start=datetime.datetime.now()
print(start)

2020-02-25 00:26:41.398487


# GridSearch for LGBM

In [10]:
params= {
    'n_estimators': [5,10],
    'max_depth': [2,4],
}

In [11]:
lgbm_clf = lgbm.LGBMClassifier()

In [12]:
grid_cv =GridSearchCV(lgbm_clf, params, cv=2, n_jobs=12)
grid_cv.fit(x_train,y_train)

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='warn', n_jobs=12,
             param_grid={'max_depth': [2, 4], 'n_estimators': [5, 10]},
             p

In [13]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'max_depth': 4, 'n_estimators': 10}
0.7922583629181459


In [14]:
best_params_ = grid_cv.best_params_

# Predict

In [15]:
lgbm_clf = lgbm.LGBMClassifier(**best_params_, n_jobs=8)

In [16]:
lgbm_clf.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=4,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=10, n_jobs=8, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [17]:
print("test acc: {:.4f}".format(lgbm_clf.score(x_test, y_test)))

test acc: 0.7932


In [21]:
pred=lgbm_clf.predict_proba(test_x)
pre =lgbm_clf.predict(test_x)

In [19]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)

now=datetime.datetime.now()
print(now)

걸린 시간: 0:01:54.789706
2020-02-25 00:28:36.188296


# Model Predict